# RNN

We use RNN as it can also remember previous prediction whereas normal NN cant remember previous 

![alt text](https://cdn-images-1.medium.com/max/800/1*vPfe01ALNgbxw8DP_4RFVw.png)

A normal NN with 1 Hidden Layer :::::: It have Different Activation layers :::: these layers are matrices 

An activation is a number calculated by layer


A CNN with 2 fully connected layers
![alt text](https://cdn-images-1.medium.com/max/800/1*VEEVatttQmlWeI98vTO0iA.png)

WE are gonna predict 3rd char from 2 input chars 

Input can be one-hot-encoded character (length of vector = # of unique characters) or a single integer and pretend it is one-hot-encoded by using an embedding layer.

  ![alt text](https://cdn-images-1.medium.com/max/800/1*gc1z1R1d5zHkYc75iqSWtw.png)
  

In [0]:
!pip uninstall fastai
!pip uninstall Pillow
!pip install Pillow==4.1.1
!pip install "fastai==0.7.0"
!pip install torchtext==0.2.3
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
!apt update && apt install -y libsm6 libxext6

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch
!pip install image

%matplotlib inline


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

In [0]:
PATH='data/nietzsche/'   # we will work on nietzche text


In [4]:
get_data("https://s3.amazonaws.com/text-datasets/nietzsche.txt", f'{PATH}nietzsche.txt')
text = open(f'{PATH}nietzsche.txt').read()
print('corpus length:', len(text))

nietzsche.txt: 606kB [00:00, 720kB/s]                            

corpus length: 600893


In [5]:
text[:400]

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to Truth, have been unskilled and unseemly methods for\nwinning a woman? Certainly she has never allowed herself '

In [13]:
chars = sorted(list(set(text))) #set will create all the unique letters and we sort them
vocab_size = len(chars)+1
print('total chars:', vocab_size) # unique letters


total chars: 85


In [0]:
chars.insert(0,"\0") #adding padding


In [10]:
''.join(chars[1:-6])

'\x00\x00\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxy'

In [0]:
char_indices = dict((c, i) for i, c in enumerate(chars)) # mapping every char to unique ID this will create an index i and take element from c
indices_char = dict((i, c) for i, c in enumerate(chars)) # mapping every ID to char

char_indices


In [0]:
indices_char

In [17]:
# creating char list of indices
idx = [char_indices[c] for c in text]
idx[:10]

[39, 41, 28, 29, 24, 26, 28, 0, 0, 0]

In [18]:
# confirming transformation of index and char
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

**Question: Character based model vs. word based model**

Generally, you want to combine character level model and word level model (e.g. for translation).

Character level model is useful when a vocabulary contains unusual words — which word level model will just treat as “unknown”. When you see a word you have not seen before, you can use a character level model.

There is also something in between that is called Byte Pair Encoding (BPE) which looks at n-gram of characters.


## 3 char model


### creating input

Now let's build a model where we can pass in 3 chars and get the fourth back. For this to work, we will need to grab the 0, 1, 2, 3 position chars in a loop.

In [0]:
# Get every nth element in indices -- our string of letters converted to indices
# range statement = get 0...len-3, skipping 3 chars at a time

cs = 3 # character skip
c1_dat = [idx[i]   for i in range(0, len(idx)-cs, cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-cs, cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-cs, cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-cs, cs)]

In [0]:
# putting data into stack

x1 = np.stack(c1_dat) 
x2 = np.stack(c2_dat) 
x3 = np.stack(c3_dat) 
y = np.stack(c4_dat)  # our output

In [0]:
n_hidden = 256

In [0]:
# The number of latent factors to create (i.e. the size of the embedding matrix)
n_fac = 42

![alt text](https://raw.githubusercontent.com/pekoto/fast.ai/master/images/rnn5.jpg)



this is what we are working on 
same color =  same matrix

In [0]:
# we will have same matrix for same colored arrow


class Char3Model(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        
        # creating embeddings with size of vocab and outputs
        self.e = nn.Embedding(vocab_size, n_fac)

        # creating linear layers l_in
        
        # The 'green arrow' from our diagram - the layer operation from input to hidden
        self.l_in = nn.Linear(n_fac, n_hidden)

        # The 'orange arrow' from our diagram - the layer operation from hidden to hidden
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        
        # The 'blue arrow' from our diagram - the layer operation from hidden to output
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, c1, c2, c3):
        in1 = F.relu(self.l_in(self.e(c1))) # each char will go through embedding->linear_layer->relu layer self.e = size 42
        in2 = F.relu(self.l_in(self.e(c2)))
        in3 = F.relu(self.l_in(self.e(c3)))
        
        h = V(torch.zeros(in1.size()).cuda()) # this is used to make *** look identical to ***2 ***3
        
        h = F.tanh(self.l_hidden(h+in1))  # ***1 it would be self.l_hidden(in1) but we made h == 0
        h = F.tanh(self.l_hidden(h+in2))  # ***2
        h = F.tanh(self.l_hidden(h+in3))  # ***3
        
        return F.log_softmax(self.l_out(h))

In [0]:
md = ColumnarModelData.from_arrays('.', [-1], np.stack([x1,x2,x3], axis=1), y, bs=512)  # whatever we pass-in in np.stack we will get forward(c1,c2,c3 position)



In [0]:
m = Char3Model(vocab_size, n_fac).cuda()  # this is std pytorch model NOT A LEARNER so using gpu

In [0]:
it = iter(md.trn_dl) # iterate through train data loader
*xs,yt = next(it) # get mini batch returns all x'es and y tensors
t = m(*V(xs))
#it is not 1hotencoded we used embedding to pretend it is
print(len(xs)) # c1,c2,c3

In [0]:
# pytorch optimizer
opt = optim.Adam(m.parameters(), 1e-2)

In [32]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      2.082338   0.632364  



[array([0.63236])]

In [0]:
set_lrs(opt, 0.001)

In [34]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.815071   0.452837  



[array([0.45284])]

### Testing the model

In [0]:
# can pass in 3 chars eg. y.[space]
def get_next(inp):
    # Convert character in the input to tensor
    indices = T(np.array([char_indices[c] for c in inp])) #turn those into int
    
    probabilities = m(*VV(indices)) # turn into variables turn into models
    
    i = np.argmax(to_np(probabilities)) # np.argmax returns index of the maximum values. convert to np array
    
    return chars[i]


In [44]:
get_next('y. ')

'I'

In [45]:
get_next('ppl')

'e'

In [46]:
get_next(' th')

'e'

In [47]:
get_next('and')


' '

## Creating our own model

In [0]:
#This is the size of our unrolled RNN.
cs=8

In [0]:
# For each of 0 through 7, create a list of every 8th character with that starting point. 
# These will be the 8 inputs to our model.
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(len(idx)-cs)]


In [0]:
# Then create a list of the next character in each of these series. This will be the labels for our model.
c_out_dat = [idx[j+cs] for j in range(len(idx)-cs)]

In [0]:
xs = np.stack(c_in_dat, axis=0)

In [52]:
xs.shape


(600885, 8)

In [0]:
y = np.stack(c_out_dat)


In [54]:
xs[:cs,:cs]


array([[39, 41, 28, 29, 24, 26, 28,  0],
       [41, 28, 29, 24, 26, 28,  0,  0],
       [28, 29, 24, 26, 28,  0,  0,  0],
       [29, 24, 26, 28,  0,  0,  0, 42],
       [24, 26, 28,  0,  0,  0, 42, 44],
       [26, 28,  0,  0,  0, 42, 44, 39],
       [28,  0,  0,  0, 42, 44, 39, 39],
       [ 0,  0,  0, 42, 44, 39, 39, 38]])

In [55]:
y[:cs]


array([ 0,  0, 42, 44, 39, 39, 38, 42])

###Create and train model

In [0]:
val_idx = get_cv_idxs(len(idx)-cs-1)

In [0]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)


In [0]:
class CharLoopModel(nn.Module):
    # This is an RNN!
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = F.relu(self.l_in(self.e(c)))
            h = F.tanh(self.l_hidden(h+inp)) # here hidden and input layers are different so we concatenate them next
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [0]:

m = CharLoopModel(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [61]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.983172   1.986563  



[array([1.98656])]

In [0]:
set_lrs(opt, 0.001)

In [63]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.687515   1.694019  



[array([1.69402])]

In [0]:
# One issue with the model above is that we are adding the input and the hidden activations (h) so far. 
# The input state represents the encoding of the characters, but h is the encoding of the series of characters so far. 
# Since these are different types of info, we want to concatenate them, not add them.
class CharLoopConcatModel(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac+n_hidden, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = torch.cat((h, self.e(c)), 1) # concatenating the hidden and in layer Input is the encoding of a character, and h is an encoding of series of characters. So adding them together, we might lose information. Let’s concatenate them instead. 
            inp = F.relu(self.l_in(inp)) # Don’t forget to change the input to match the shape (n_fac+n_hidden instead of n_fac).
            h = F.tanh(self.l_hidden(inp))  # replacing h with new hidden state
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [0]:
m = CharLoopConcatModel(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [0]:
it = iter(md.trn_dl)
*xs,yt = next(it)
t = m(*V(xs))

In [67]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.863278   1.841863  



[array([1.84186])]

In [0]:
set_lrs(opt, 1e-4)

In [69]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.751296   1.760035  



[array([1.76004])]

###Testing model

In [0]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [71]:
get_next('for thos')


'e'

In [72]:
get_next('part of ')

't'

In [73]:
get_next('queens a')

'n'

##RNN with pyTorch

In [0]:
# we are doing same thing as above only it is wrapped in here
# PyTorch will write the for loop automatically for us and also the linear input layer.

class CharRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h) # it returns all ellipses/ activations so we only use 1 using dim = -1
        # The minor difference in PyTorch is that self.rnn will append a new hidden state to a tensor instead of replacing 
        # (in other words, it will give back all ellipses in the diagram) . We only want the final one so we do outp[-1]
        
        return F.log_softmax(self.l_out(outp[-1]), dim=-1)

In [0]:
m = CharRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [0]:
it = iter(md.trn_dl)
*xs,yt = next(it)

In [78]:
t = m.e(V(torch.stack(xs)))
t.size()

torch.Size([8, 512, 42])

In [79]:
ht = V(torch.zeros(1, 512,n_hidden))
outp, hn = m.rnn(t, ht)
outp.size(), hn.size()

(torch.Size([8, 512, 256]), torch.Size([1, 512, 256]))

In [80]:
t = m(*V(xs)); t.size()

torch.Size([512, 85])

In [81]:

fit(m, md, 4, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.859717   1.845479  
    1      1.674511   1.672501  
    2      1.583756   1.601259  
    3      1.539196   1.557011  



[array([1.55701])]

In [0]:
set_lrs(opt, 1e-4)

In [83]:
fit(m, md, 2, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.462015   1.512779  
    1      1.470439   1.507218  



[array([1.50722])]

### Testing the model

In [0]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [85]:
get_next('for thos')

'e'

In [0]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [87]:
get_next_n('for thos', 40)

'for those and the same the same the same the sam'

##Multi-output RNN

In [0]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(0, len(idx)-cs-1, cs)]

In [0]:
c_out_dat = [[idx[i+j] for i in range(cs)] for j in range(1, len(idx)-cs, cs)]

In [95]:
xs = np.stack(c_in_dat)
xs.shape

(75111, 8)

In [96]:

ys = np.stack(c_out_dat)
ys.shape

(75111, 8)

In [97]:
xs[:cs,:cs]

array([[39, 41, 28, 29, 24, 26, 28,  0],
       [ 0,  0, 42, 44, 39, 39, 38, 42],
       [32, 37, 30,  1, 72, 60, 53, 72],
       [ 1, 43, 70, 73, 72, 60,  1, 61],
       [71,  1, 53,  1, 75, 67, 65, 53],
       [66,  8,  8, 75, 60, 53, 72,  1],
       [72, 60, 57, 66, 23,  1, 32, 71],
       [ 1, 72, 60, 57, 70, 57,  1, 66]])

In [98]:
ys[:cs,:cs]

array([[41, 28, 29, 24, 26, 28,  0,  0],
       [ 0, 42, 44, 39, 39, 38, 42, 32],
       [37, 30,  1, 72, 60, 53, 72,  1],
       [43, 70, 73, 72, 60,  1, 61, 71],
       [ 1, 53,  1, 75, 67, 65, 53, 66],
       [ 8,  8, 75, 60, 53, 72,  1, 72],
       [60, 57, 66, 23,  1, 32, 71,  1],
       [72, 60, 57, 70, 57,  1, 66, 67]])

###create and train the model

In [0]:
val_idx = get_cv_idxs(len(xs)-cs-1)


In [0]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, ys, bs=512)


In [0]:
# we reduce redundancy and improve efficiency by removing 
# reccuring parts of data and taking non overlapping set of
# characters
class CharSeqRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)    # ***4
        return F.log_softmax(self.l_out(outp), dim=-1)

Notice that we are no longer doing outp[-1] since we want to keep all of them. But everything else is identical. One complexity is that we want to use the negative log-likelihood loss function as before, but it expects two rank 2 tensors (two mini-batches of vectors). But here, we have rank 3 tensor: (rank =  number of directions req to describe it)

8 characters (time steps)
84 probabilities
for 512 minibatch


"***4" self.rnn(inp, h) is a loop applying the same matrix multiply again and again. If that matrix multiply tends to increase the activations each time, we are effectively doing that to the power of 8 — we call this a gradient explosion. We want to make sure the initial l_hidden will not cause our activations on average to increase or decrease.

A nice matrix that does exactly that is called identity matrix:

In [0]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [0]:
it = iter(md.trn_dl)
*xst,yt = next(it)

In [0]:
# Custom Loss Function

def nll_loss_seq(inp, targ):
    sl,bs,nh = inp.size()
    targ = targ.transpose(0,1).contiguous().view(-1)
    return F.nll_loss(inp.view(-1,nh), targ)

F.nll_loss is the PyTorch loss function.
Flatten our inputs and targets.

Transpose the first two axes because PyTorch expects 
1. sequence length (how many time steps), 
2. batch size, 
3. hidden state itself. yt.size() is 512 by 8, whereas sl, bs is 8 by 512.

PyTorch does not generally actually shuffle the memory order when you do things like ‘transpose’, but instead it keeps some internal metadata to treat it as if it is transposed.

When you transpose a matrix, PyTorch just updates the metadata . If you ever see an error that says “this tensor is not continuous” , add .contiguous() after it and error goes away.
.view is same as np.reshape. -1 indicates as long as it needs to be.

In [106]:
fit(m, md, 4, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      2.611669   2.415422  
    1      2.298023   2.204125  
    2      2.143688   2.090011  
    3      2.046707   2.014779  



[array([2.01478])]

Remember that fit(...) is the lowest level fast.ai abstraction that implements the training loop. So all the arguments are standard PyTorch things except for md which is our model data object which wraps up the test set, the training set, and the validation set.

###Identity Init

In [0]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [108]:
m.rnn.weight_hh_l0.data.copy_(torch.eye(n_hidden)) # We can overwrite the randomly initialized hidden-hidden weight with an identity matrix:



    1     0     0  ...      0     0     0
    0     1     0  ...      0     0     0
    0     0     1  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      1     0     0
    0     0     0  ...      0     1     0
    0     0     0  ...      0     0     1
[torch.cuda.FloatTensor of size 256x256 (GPU 0)]

In [109]:
fit(m, md, 4, opt, nll_loss_seq)


epoch      trn_loss   val_loss   
    0      2.424316   2.265299  
    1      2.156431   2.096609  
    2      2.038416   2.011768  
    3      1.976292   1.959522  



[array([1.95952])]

In [0]:
set_lrs(opt, 1e-3)


In [111]:
fit(m, md, 4, opt, nll_loss_seq)


epoch      trn_loss   val_loss   
    0      1.881031   1.891031  
    1      1.872315   1.883545  
    2      1.859521   1.876229  
    3      1.853916   1.871369  



[array([1.87137])]